In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import feature_spaces
import pandas as pd
from typing import List
from os.path import join
import numpy as np
import encoding_utils, feature_spaces
import pickle as pkl
from collections import defaultdict
from feature_spaces import *
NUM_VOXELS = 250

In [2]:
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])
wordseqs = feature_spaces.get_story_wordseqs(test_stories)

In [3]:
def get_words_for_story(wordseq, max_running_words=3) -> List[str]:
    running_words = []
    
    words = np.array(wordseq.data)
    TRIM = 5
    tr_times = wordseq.tr_times[5+TRIM: -TRIM]
    for i, tr_time in enumerate(tr_times):
        valid_times = wordseq.data_times <= tr_time
        # print(valid_times)
        running_words.append(' '.join(words[valid_times][-max_running_words:]))
    #     print(tr_time, running_words)
    return running_words

texts_list_test = []
for story_name in tqdm(test_stories):
    wordseq = wordseqs[story_name]
    texts_list_test.append(get_words_for_story(wordseq))
texts_test = sum(texts_list_test, [])

100%|██████████| 1/1 [00:00<00:00, 307.34it/s]


In [4]:
subj = 'UTS03'

# select top_idxs
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03'
corrs_val = np.load(join(save_dir, 'corrs.npz'))['arr_0']
top_idxs = np.argsort(corrs_val)[::-1][:NUM_VOXELS]
print(corrs_val[top_idxs][:5])

# load responses (n_time_points x n_voxels)
resp_test = encoding_utils.get_response(test_stories, subj)

[0.76296981 0.74258237 0.72107898 0.71616266 0.71508206]


In [6]:
# get the top ngrams for each voxel
NUM_NGRAMS = 30
top_ngrams = {}
for i, voxel in enumerate(tqdm(top_idxs)):
    top_resp_idxs = np.argsort(resp_test[:, i])[::-1]
    top_ngrams[voxel] = [texts_test[idx][:NUM_NGRAMS] for idx in top_resp_idxs]
top_ngrams = pd.DataFrame.from_dict(top_ngrams)
top_ngrams.columns = [f'voxel_top_{i}' for i in range(NUM_VOXELS)]

100%|██████████| 250/250 [00:00<00:00, 18841.32it/s]


In [7]:
top_ngrams.head(n=5)

,voxel_top_0,voxel_top_1,voxel_top_2,voxel_top_3,voxel_top_4,voxel_top_5,voxel_top_6,voxel_top_7,voxel_top_8,voxel_top_9,...,voxel_top_240,voxel_top_241,voxel_top_242,voxel_top_243,voxel_top_244,voxel_top_245,voxel_top_246,voxel_top_247,voxel_top_248,voxel_top_249
0,have my cigarettes,of his cigarette,and he sees,of light and,person comes closer,our thirty second,see her again,she had horses,you a ride,this suburban street,...,i ran i,i had no,at least once,want any part,the girl reaches,the neighborhood this,cigarettes for me,both need this,does the opposite,driver's license yet
1,see me and,you know let's,you know let's,pulls out a,and i walk,when i see,in the front,like we would,one match inside,cigarette and we,...,wider and wider,she says never,still understands how,someone who has,say you got,i was crying,that was close,i'm a waitress,someone who has,carl's efficiency apartments
2,now to my,at home this,want any part,be friends i,and emergecenters and,and i walk,pulls out a,or lonely or,and i follow,when i see,...,and i say,i know it's,there is one,checking and it's,want any part,was ok because,our cigarettes she,one bedroom apartment,seven cigarettes on,have my cigarettes
3,person comes closer,checking and it's,i'm not a,be we are,checking and it's,be friends i,very very bad,i ran i,we share some,and he says,...,walking and uh,still understands how,i ran i,great to meet,i'm a waitress,want any part,leads me to,learn this time,be friends i,i had no
4,i'm going home,was fun and,cigarette and we,me going she,has fifty million,saw you yeah,when i see,has fifty million,and i walk,who the fuck,...,i had no,ever tell people,i was crying,what our lives,i think no,stories about our,going on there's,be real nice,had my cigarettes,was gonna happen


In [8]:
top_ngrams.to_pickle('top_ngrams.pkl')